In [1]:
import os
import cv2
import numpy as np
import tensorflow.keras as keras
import tensorflow as tf
from tensorflow.keras import layers,Sequential
from tensorflow.keras.layers import Lambda,Dense,ReLU,MaxPooling2D,Flatten
from tensorflow.keras.layers import Dense,Input,MaxPool2D,Conv2D,Dropout,Add
import matplotlib.pyplot as plt
from utils import *
from tensorflow.keras.models import Model

from tensorflow.keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

In [2]:
num_classes = 10
input_shape = (28, 28, 1)
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255

x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [3]:
inputs = Input(shape=(28, 28, 1))
output_1 = Involution(channel=3, group_number=1, kernel_size=3, stride=1, reduction_ratio=2, name="involution_1")(inputs)
output_1 = ReLU()(output_1)
output_1 = Add()([output_1, inputs])

output_2 = Involution(channel=3, group_number=1, kernel_size=3, stride=1, reduction_ratio=2, name="involution_2")(output_1)
output_2 = ReLU()(output_2)
output_2 = Add()([output_2, output_1])

output_1 = Flatten()(output_1)
output_1 = Dropout(0.5)(output_1)
output_1 = Dense(num_classes, activation="softmax")(output_1)
model = Model(inputs=inputs, outputs=output_1)

model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
involution_1 (Involution)       (None, 28, 28, 1)    24          input_1[0][0]                    
__________________________________________________________________________________________________
re_lu (ReLU)                    (None, 28, 28, 1)    0           involution_1[0][0]               
__________________________________________________________________________________________________
add (Add)                       (None, 28, 28, 1)    0           re_lu[0][0]                      
                                                                 input_1[0][0]         

In [4]:
batch_size = 128
epochs = 20
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Epoch 1/20
422/422 [==============================] - 5s 12ms/step - loss: 0.8209 - accuracy: 0.7475 - val_loss: 0.3163 - val_accuracy: 0.9187
Epoch 2/20
422/422 [==============================] - 5s 11ms/step - loss: 0.4860 - accuracy: 0.8607 - val_loss: 0.2784 - val_accuracy: 0.9232
Epoch 3/20
422/422 [==============================] - 5s 11ms/step - loss: 0.4488 - accuracy: 0.8702 - val_loss: 0.2631 - val_accuracy: 0.9295
Epoch 4/20
422/422 [==============================] - 4s 11ms/step - loss: 0.4304 - accuracy: 0.8753 - val_loss: 0.2584 - val_accuracy: 0.9287
Epoch 5/20
422/422 [==============================] - 5s 11ms/step - loss: 0.4228 - accuracy: 0.8773 - val_loss: 0.2521 - val_accuracy: 0.9317
Epoch 6/20
422/422 [==============================] - 4s 11ms/step - loss: 0.4188 - accuracy: 0.8784 - val_loss: 0.2521 - val_accuracy: 0.9310
Epoch 7/20
422/422 [==============================] - 5s 11ms/step - loss: 0.4143 - accuracy: 0.8797 - val_loss: 0.2454 - val_accuracy: 0.9328

In [5]:
score = model.evaluate(x_test, y_test, verbose=0)
score

[0.2921508550643921, 0.9193000197410583]